<a href="https://www.kaggle.com/code/tarekyahia/generative-question-answer-system-with-haystack?scriptVersionId=168570482" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### besmi allah

## Libraries 

In [1]:
# install haystack
!pip install --upgrade pip

!pip install \
peft \
evaluate==0.4.0 \
rouge_score==0.1.2 \

!pip install 'farm-haystack[all]' ## or 'all-gpu' for the GPU-enabled dependencies


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f8b787ecafdc50229d624f01adae6a3424d9002173662b1af40e297b5e20a925
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

In [2]:
import os
import numpy as np 
import pandas as pd
import re


from haystack.utils import convert_files_to_docs
from haystack.nodes import PreProcessor
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.pipelines import Pipeline
from haystack.nodes.sampler import TopPSampler
from haystack.nodes.ranker import LostInTheMiddleRanker

from evaluate import load
metric = load("rouge")

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer,AutoTokenizer
import torch
from peft import PeftModel, PeftConfig


import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)



2024-03-24 12:58:44.131639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 12:58:44.131823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 12:58:44.313689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 12:59:04,237	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `

## Knowledge Base :

In [3]:

BOOK_DIR = '/kaggle/input/mlbookspdfs' # to the books

# show the books 
for dirname, _, filenames in os.walk(BOOK_DIR):
    print('-------------------------------------------')
    for filename in filenames:
        print(filename)

-------------------------------------------
mml-book.pdf
Ng_MachineLearningYearning.pdf
Deep Learning by Ian Goodfellow, Yoshua Bengio, Aaron Courville (z-lib.org).pdf


## Preprocessing

- PDFs to TXT and cleaning

In [4]:
def clean_text(text: str) -> str:
    """
    
   Note: The source code for the function is a combination of clean_wiki_text (available in haystack) and some custom cleanup.
   
    """
    # get rid of multiple new lines
    while "\n\n" in text:
        text = text.replace("\n\n", "\n")

    # remove extremely short lines
    lines = text.split("\n")
    cleaned = []
    for l in lines:
        if len(l) > 30 or (l[:2] == "==" and l[-2:] == "=="):
            cleaned.append(l)
    text = "\n".join(cleaned)

    # add paragraphs (identified by wiki section title which is always in format "==Some Title==")
    text = text.replace("\n==", "\n\n\n==")

    # remove empty paragrahps
    text = re.sub(r"(==.*==\n\n\n)", "", text)
    
    # custom
    
    # Remove "Table of Contents"
    text = re.sub(r'Table of Contents', '', text, flags=re.IGNORECASE)  
    
    # Remove patterns like 4\x0c
    text = re.sub(r'\b\d*\x0c\b', '', text)  
    
    # Remove patterns like /n1, and /n
    text = re.sub(r'(\\n|\n)\d+|/n', '', text)  # Remove patterns like /n1, /nNUM, and /n
    
    # Replace newline characters with spaces that between words
    text = text.replace('\n', '')
    
    # remove #\u200b
    text = text.replace('\u200b', '')  
    

    return text

In [5]:
#all_docs = convert_files_to_docs(dir_path="/kaggle/input/ng-machinelearningyearning", clean_func=clean_text)

all_docs = convert_files_to_docs(dir_path=BOOK_DIR,clean_func=clean_text)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [6]:
print(len(all_docs))

3


- split to chunks and some more cleaning

In [7]:
processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    remove_substrings=None,
    split_by="word",
    split_length=100, # suitable for the dense vector
    split_respect_sentence_boundary=True,
    split_overlap= 4,
    max_chars_check = 10_000,
    progress_bar = True
)
docs = processor.process(all_docs)

Preprocessing: 100%|██████████| 3/3 [00:02<00:00,  1.01docs/s]


In [8]:
print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

n_files_input: 3
n_docs_output: 5982


In [9]:
# look at example
sample = docs[5]
sample
# 4\x0c
# assume\nthat

<Document: {'content': 'However, this brings with it the dangerthat a practitioner becomes unaware of the design decisions and, hence,the limits of machine learning algorithms.The enthusiastic practitioner who is interested to learn more about themagic behind successful machine learning algorithms currently faces adaunting set of pre-requisite knowledge:Programming languages and data analysis toolsLarge-scale computation and the associated frameworksMathematics and statistics and how machine learning builds on itAt universities, introductory courses on machine learning tend to spendearly parts of the course covering some of these pre-requisites. For histori-cal reasons, courses in machine learning tend to be taught in the computerscience department, where students are often trained in the first two areasof knowledge, but not so much in mathematics and statistics.Current machine learning textbooks primarily focus on machine learn-ing algorithms and methodologies and assume that the read

##  retrieval
    - using FAISS DB to work more with embedings

In [10]:
document_store = FAISSDocumentStore(sql_url="sqlite:///", faiss_index_factory_str="Flat")

# write the docs to the DB
document_store.write_documents(docs)

Writing Documents: 10000it [00:20, 490.76it/s]             


In [11]:
retriever = EmbeddingRetriever(
    document_store=document_store, 
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    max_seq_len = 256,
    batch_size = 32,
    top_k = 10
)
# Important:
# Now that we initialized the Retriever, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on the corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it to the existing document embeddings, which is very fast.
document_store.update_embeddings(retriever)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Updating Embedding:   0%|          | 0/5982 [00:00<?, ? docs/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Documents Processed: 10000 docs [33:44,  4.94 docs/s]           


## Generative

In [12]:
model_name = 'google/t5-v1_1-base' #'facebook/bart-large-cnn'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype = torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## RAG Integration

In [13]:
# pipeline


def model_predict(model,prompt):
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)#, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def RAG_pipeline(question):

    pipeline = Pipeline()
    pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
    pipeline.add_node(component=TopPSampler(top_p=0.90), name="Sampler", inputs=["Retriever"])
    pipeline.add_node(component=LostInTheMiddleRanker(1024), name="LostInTheMiddleRanker", inputs=["Sampler"])
    return pipeline.run(query=question)
    
    
prompts = []    
    
def RAG_Predict(model, question):
 
    result = RAG_pipeline(question)
    
    context = ""
    for d in result['documents']:
        context += d.content
    context
    
    prompt = f"""Answer the following question based on the context: \n Question: {question} \n Context: \n {context}  \n Answer: """
    prompts.append(prompt)
    
    return model_predict(model,prompt)


## test zero shot

In [14]:
# look at the output before finetuning
questions = [
    "What is machine learning?",
    "What is the difference between supervised and unsupervised learning?",
    "What is a neural network?",
    "What is the purpose of activation functions in neural networks?",
    "What is overfitting in machine learning?",
    "What is gradient descent?",
    "What is the difference between classification and regression in machine learning?",
    "What is a convolutional neural network (CNN) used for?",
    "What is transfer learning in deep learning?",
    "What is the purpose of regularization techniques in machine learning?"
]

In [15]:
# 10 questions

# taking answers
answers = []

for question in questions:
    
    answer = RAG_Predict(model,question)
    answers.append(answer)
    
 # df   
base_answers = pd.DataFrame({'questions' : questions, 'answers' : answers})

pd.set_option('display.max_colwidth', None)
base_answers

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,questions,answers
0,What is machine learning?,: Question: What is machine learning? Context: 1Context: 1Context
1,What is the difference between supervised and unsupervised learning?,supervised learning is a type of machine learning. supervised learning is
2,What is a neural network?,based on the context: Context: Biological neurons are the most important type of
3,What is the purpose of activation functions in neural networks?,function. Answer: Yes. This is the default activation function. Answer: Yes.
4,What is overfitting in machine learning?,", you can overfit the training set. Answer: Yes, you can overfit"
5,What is gradient descent?,:
6,What is the difference between classification and regression in machine learning?,: Context: In this chapter we will discuss classification.: Support vector machines.
7,What is a convolutional neural network (CNN) used for?,. Answer: Convolution is a specialized kind of linear operation. is
8,What is transfer learning in deep learning?,: Transfer learning in deep learning: Transfer learning in deep learning: Transfer learning in
9,What is the purpose of regularization techniques in machine learning?,a a s a s a


## evaluate:

In [16]:
# chat gpt answers -> i used the same prompt with the same context
# to evaluate with rouge and semantic simalarty

chatGPT_answers = [
    "Machine learning is about designing algorithms that automatically extract valuable information from data. It emphasizes automatic extraction of meaningful information from data without much domain-specific expertise, aiming for general-purpose methodologies applicable to various datasets.",
    "Supervised learning involves observing several examples of a random vector x and an associated value or vector y, and learning to predict y from x. In contrast, unsupervised learning involves observing several examples of a random vector x without associated values and attempting to implicitly or explicitly learn the probability distribution p(x) or some interesting properties of that distribution.",
    "Neural networks, specifically feedforward networks, are composed of interconnected functions represented by a directed acyclic graph. They are organized into layers, with each layer being a function of the preceding one. Hidden unit design and determining the architecture are key considerations in neural network design.",
    "The activation function in neural networks, such as the rectified linear unit (ReLU), is applied element-wise to the output of linear transformations. It introduces nonlinearity to the network, enabling it to learn complex patterns and relationships in the data.",
    "Overfitting in machine learning occurs when a model memorizes the training data excessively, leading to poor performance on unseen data (test set). It happens when the model has high capacity and fits the noise in the training data rather than capturing the underlying patterns.",
    "Gradient descent is a first-order optimization algorithm used to find a local minimum of a function by taking steps proportional to the negative gradient of the function at the current point. Stochastic gradient descent is an extension that uses a subset of training examples in each iteration to approximate the gradient.",
    "Classification involves predicting discrete labels or categories for input data, while regression involves predicting continuous values. In classification, the labels are typically integers, whereas in regression, the labels are real-valued.",
    "Convolutional neural networks (CNNs) are specialized neural networks for processing data with a known grid-like topology, such as time-series or image data. They utilize convolution, a specialized kind of linear operation, to extract features from the input data efficiently.",
    "Transfer learning in deep learning involves leveraging knowledge learned from one task or domain to improve performance on another related task or domain. It can reduce the amount of labeled data needed for training and enhance generalization to new tasks.",
    "Regularization techniques in machine learning are designed to reduce generalization error (error on unseen data) without significantly increasing training error. They include adding constraints or penalties on model parameters to prevent overfitting and improve model performance on unseen data. Regularization is crucial for controlling the complexity of models and preventing overfitting."
]


# rouge function

def eval_rouge(refs,preds):
    
    result = metric.compute(predictions=preds, references=refs, use_stemmer=True, use_aggregator=False)
    
    return result['rouge1'],np.mean(result['rouge1'])
print(eval_rouge(chatGPT_answers,answers))

([0.13636363636363638, 0.16901408450704225, 0.13793103448275862, 0.15686274509803924, 0.14545454545454545, 0.0, 0.09523809523809523, 0.2745098039215686, 0.22641509433962265, 0.0], 0.13417890394053084)


In [17]:
# similarty 
from sentence_transformers import SentenceTransformer, util

def cos_sim(refs,preds):
    

    model = SentenceTransformer("all-MiniLM-L6-v2")

    # Encode all sentences
    refs_embd = model.encode(refs)
    preds_embd = model.encode(preds)

    # Compute cosine similarity between all pairs
    cos_sim = util.cos_sim(refs_embd, preds_embd)
    
    return cos_sim



In [18]:
#instructed model

# similarity
sim = []
for r,q in zip(chatGPT_answers,answers):
    sim.append(float(cos_sim(r,q)[0, 0]))
print(f" the average of the similarity =  {np.mean(sim)}")


rouges,ave = eval_rouge(chatGPT_answers,answers)

print(f" the average of the rouge scores is  =  {ave} \n the rouge scores = {rouges}")

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 the average of the similarity =  0.4232235012575984
 the average of the rouge scores is  =  0.13417890394053084 
 the rouge scores = [0.13636363636363638, 0.16901408450704225, 0.13793103448275862, 0.15686274509803924, 0.14545454545454545, 0.0, 0.09523809523809523, 0.2745098039215686, 0.22641509433962265, 0.0]


## LoRa Fine tune

[fine-tune notebook](https://www.kaggle.com/code/tarekyahia/fine-tune-flan-t5-question-answer-squad/notebook)

In [19]:
# load

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       '/kaggle/input/peft-flan-t5/checkpoint-29982', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [20]:
answers_fine = []

for question in questions:
    answer = RAG_Predict(peft_model,question)
    answers_fine.append(answer)
    
# 10 questions

fine_answers = pd.DataFrame({'questions' : questions, 'answers' : answers_fine})

fine_answers

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,questions,answers
0,What is machine learning?,designing algorithms that automatically extractvaluable information from data
1,What is the difference between supervised and unsupervised learning?,There is noobjective test for distinguishing whether a value is a feature or
2,What is a neural network?,neural networks are called networks because they are typicallyrepresented by composing together many different functions
3,What is the purpose of activation functions in neural networks?,Applyingthis function to the output of a linear transformation yields a nonlinear
4,What is overfitting in machine learning?,gap between the training error and test error
5,What is gradient descent?,first-order optimization algorithm
6,What is the difference between classification and regression in machine learning?,classification are integers
7,What is a convolutional neural network (CNN) used for?,"processing datathat has a known, grid-like topology"
8,What is transfer learning in deep learning?,multi-task learning
9,What is the purpose of regularization techniques in machine learning?,reduce the test error


In [21]:
print(eval_rouge(chatGPT_answers,answers_fine))

([0.3181818181818182, 0.1095890410958904, 0.3, 0.30769230769230765, 0.15384615384615385, 0.14285714285714288, 0.17647058823529413, 0.24489795918367344, 0.09302325581395349, 0.10909090909090909], 0.19556491759971434)


In [22]:
# fine tuned model

# similarity
sim = []
for r,q in zip(chatGPT_answers,answers_fine):
    sim.append(float(cos_sim(r,q)[0, 0]))
print(f" the average of the similarity =  {np.mean(sim)}")


rouges,ave = eval_rouge(chatGPT_answers,answers)

print(f" the average of the rouge scores is  =  {ave} \n the rouge scores = {rouges}")

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 the average of the similarity =  0.4192570194602013
 the average of the rouge scores is  =  0.13417890394053084 
 the rouge scores = [0.13636363636363638, 0.16901408450704225, 0.13793103448275862, 0.15686274509803924, 0.14545454545454545, 0.0, 0.09523809523809523, 0.2745098039215686, 0.22641509433962265, 0.0]
